In [ ]:
#설치
%%capture --no-stderr
%pip install langchain_community langchainhub chromadb langchain langgraph tavily-python langchain-text-splitters langchain_openai

In [ ]:
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
import os

llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0)

#Tavily
tavily = TavilyClient(api_key='')

In [ ]:
import os
from pprint import pprint
from typing import List, TypedDict

from langchain_commTavilyClientunity.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import END, StateGraph
from tavily import TavilyClient


# -----------------------------------------------------------------
# INDEXING
# -----------------------------------------------------------------

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# ChromaDB를 사용한 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)
retriever = vectorstore.as_retriever()


# -----------------------------------------------------------------
#  LangGraph Node 설정
# -----------------------------------------------------------------
class GraphState(TypedDict):
    """
    그래프의 상태를 나타냅니다.

    Attributes:
        question: 사용자의 질문
        generation: LLM이 생성한 답변
        documents: 검색된 문서 목록
        retries: 답변 재시도 횟수
        web_searched: 웹 검색 수행 여부 # MODIFIED: 웹 검색 루프 방지를 위한 상태 추가
    """
    question: str
    generation: str
    documents: List[Document]
    retries: int
    web_searched: bool # MODIFIED


def retrieve(state):
    """벡터 저장소에서 문서를 검색합니다."""
    print("--- 1. 문서 검색 (RETRIEVE) ---")
    question = state["question"]
    documents = retriever.invoke(question)
    print(f"검색된 문서 수: {len(documents)}")
    # MODIFIED: 상태 초기화
    return {"documents": documents, "question": question, "retries": 0, "web_searched": False}


def grade_documents(state):
    """검색된 문서가 질문과 관련이 있는지 평가합니다."""
    print("--- 2. 관련성 확인 (GRADE DOCUMENTS) ---")
    question = state["question"]
    documents = state["documents"]

    system = """You are a grader assessing relevance of a retrieved document to a user question.
    If the document contains keywords related to the user question, grade it as relevant.
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "question: {question}\n\n document: {document} "),
    ])
    retrieval_grader = prompt | llm | JsonOutputParser()

    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score["score"]
        if grade.lower() == "yes":
            print("  - GRADE: 문서 관련성 있음")
            filtered_docs.append(d)
        else:
            print("  - GRADE: 문서 관련성 없음")

    state['documents'] = filtered_docs
    return state


def web_search(state):
    """Tavily를 사용하여 웹 검색을 수행합니다."""
    print("--- 4a. 웹 검색 (WEB SEARCH) ---")
    question = state["question"]

    print(f"웹 검색 질의: {question}")
    response = tavily.search(query=question, max_results=3)
    web_results = [
        Document(
            page_content=obj["content"],
            metadata={"source": obj["url"], "title": obj["title"]}
        ) for obj in response['results']
    ]
    print(f"웹 검색 결과 수: {len(web_results)}")
    state['documents'] = web_results
    state['web_searched'] = True # MODIFIED: 웹 검색 수행 플래그 설정
    return state


def generate(state):
    """검색된 문서를 바탕으로 답변을 생성합니다."""
    print("--- 3. 답변 생성 (GENERATE) ---")
    question = state["question"]
    documents = state["documents"]

    sources = "\n".join([f"Source URL: {doc.metadata.get('source', 'N/A')}, Title: {doc.metadata.get('title', 'N/A')}" for doc in documents])

    system = f"""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Your final answer must include the sources (URL and title) provided below.

    Sources:
    {sources}
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Question: {question}\n\nContext: {context}"),
    ])
    rag_chain = prompt | llm | StrOutputParser()

    generation = rag_chain.invoke({"context": "\n\n".join([d.page_content for d in documents]), "question": question})
    state["generation"] = generation
    return state


def hallucination_checker(state):
    """생성된 답변에 환각(hallucination)이 있는지 평가합니다."""
    print("--- 5. 환각 확인 (HALLUCINATION CHECKER) ---")
    documents = state["documents"]
    generation = state["generation"]

    system = """You are a grader assessing whether an answer is grounded in / supported by a set of facts.
    Give a binary 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts.
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "Documents: {documents}\n\nAnswer: {generation}"),
    ])
    hallucination_grader = prompt | llm | JsonOutputParser()

    score = hallucination_grader.invoke({"documents": [d.page_content for d in documents], "generation": generation})
    grade = score["score"]

    if grade == "yes":
        print("  - CHECK: 환각 없음. 답변 완료.")
        return "useful"
    else:
        print("  - CHECK: 환각 감지. 재시도.")
        state["retries"] = state.get("retries", 0) + 1
        return "not_supported"


def handle_failure(state):
    """관련성 또는 환각 문제로 실패 시 처리합니다."""
    print("--- 최종 실패 처리 ---")
    retries = state.get("retries", 0)
    web_searched = state.get("web_searched", False)

    if not state.get("documents") and web_searched:
        generation = "failed: not relevant"
    elif retries >= 1:
        generation = "failed: hallucination"
    else:
        generation = "failed: unknown error"

    return {"generation": generation}


# -----------------------------------------------------------------
# LangGraph 엣지 (흐름 제어) 정의
# -----------------------------------------------------------------

def decide_to_generate(state):
    """문서의 관련성 여부에 따라 답변 생성 또는 웹 검색으로 분기합니다."""
    print("--- 2a. 관련성 기반 분기 ---")
    web_searched = state.get("web_searched", False)

    if not state.get("documents"):
        if web_searched:
            # MODIFIED: 웹 검색을 이미 했는데도 관련 문서가 없으면 실패 처리
            print("  - DECISION: 웹 검색 후에도 관련 문서 없음 -> 실패 처리로 이동")
            return "failure"
        else:
            # MODIFIED: 관련 문서가 없으면 웹 검색으로 이동
            print("  - DECISION: 관련 문서 없음 -> 웹 검색으로 이동")
            return "web_search"
    else:
        print("  - DECISION: 관련 문서 있음 -> 답변 생성으로 이동")
        return "generate"


def check_hallucination_and_retry(state):
    """환각 여부 및 재시도 횟수에 따라 분기합니다."""
    print("--- 5a. 환각 기반 분기 ---")
    decision = hallucination_checker(state)
    retries = state.get("retries", 0)

    if decision == "useful":
        return "end"
    elif retries < 1:
        print(f"  - 재시도 횟수: {retries}. 답변 재생성.")
        return "retry"
    else:
        print(f"  - 재시도 횟수: {retries}. 실패 처리.")
        return "failure"


# -----------------------------------------------------------------
# build
# -----------------------------------------------------------------

workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("web_search", web_search)
workflow.add_node("generate", generate)
workflow.add_node("handle_failure", handle_failure)

# 엣지(흐름) 설정
workflow.set_entry_point("retrieve") # MODIFIED: 시작점을 retrieve로 변경
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
        "failure": "handle_failure", # MODIFIED: 실패 경로 추가
    },
)
workflow.add_edge("web_search", "grade_documents")
workflow.add_conditional_edges(
    "generate",
    check_hallucination_and_retry,
    {
        "retry": "generate",
        "failure": "handle_failure",
        "end": END,
    },
)
workflow.add_edge("handle_failure", END)

# 그래프 컴파일
app = workflow.compile()


In [ ]:
# Test Case 1: Vector Store에 관련 정보가 있는 경우
print("\n\n--- [Test Case 1] 실행: 'What is prompt?' ---\n")
inputs = {"question": "What is prompt?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"노드: {key}")
pprint(f"최종 답변:\n{value['generation']}")


# Test Case 2: Vector Store에 관련 정보가 없어 웹 검색을 수행하는 경우
print("\n\n--- [Test Case 2] 실행: 'What is KAKAO CORP?' ---\n")
inputs = {"question": "What is KAKAO CORP?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"노드: {key}")
pprint(f"최종 답변:\n{value['generation']}")

# Test Case 3: 웹 검색 후에도 관련 정보가 없는 경우 (Tavily 검색어 조정)
print("\n\n--- [Test Case 3] 실행: '96125m600' ---\n")
inputs = {"question": "asdlfkjasdlfkjaskldfj"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"노드: {key}")
pprint(f"최종 답변:\n{value['generation']}")



--- [Test Case 1] 실행: 'What is prompt?' ---

--- 1. 문서 검색 (RETRIEVE) ---
검색된 문서 수: 4
'노드: retrieve'
--- 2. 관련성 확인 (GRADE DOCUMENTS) ---
  - GRADE: 문서 관련성 있음
  - GRADE: 문서 관련성 있음
  - GRADE: 문서 관련성 있음
  - GRADE: 문서 관련성 있음
--- 2a. 관련성 기반 분기 ---
  - DECISION: 관련 문서 있음 -> 답변 생성으로 이동
'노드: grade_documents'
--- 3. 답변 생성 (GENERATE) ---
--- 5a. 환각 기반 분기 ---
--- 5. 환각 확인 (HALLUCINATION CHECKER) ---
  - CHECK: 환각 없음. 답변 완료.
'노드: generate'
('최종 답변:\n'
 'A prompt is a sequence of prefix tokens that increases the probability of '
 'obtaining a desired output given an input. These prompts can be treated as '
 'trainable parameters and optimized directly in the embedding space using '
 'techniques like gradient descent. Various methods such as AutoPrompt, '
 'Prefix-Tuning, P-tuning, and Prompt-Tuning have been developed to enhance '
 'the effectiveness of prompts in generating outputs. Additionally, Automatic '
 'Prompt Engineering (APE) is a method that searches through model-generated '
 'instruct